In [32]:
import numpy as np
import pandas as pd
import nibabel as nib
from matplotlib import pyplot as plt
import scipy.stats as sc
import conf
import importlib
import os
importlib.reload(conf)

<module 'conf' from '/Volumes/homes/Maya/Guests/Amit/CompSagolProj/conf.py'>

In [33]:
# Calculate mean voxel activity given a specified voxel index
def get_region_mean(region_idx, data, atlas_cifti_data):
    region_vertices_mask = atlas_cifti_data == region_idx
    region_timeseries = np.mean(data[:,region_vertices_mask[0]], axis = 1)
    #print(f'shape of region_timeseries: {region_timeseries}')
    return region_timeseries

In [34]:
# Extract list of subjects with the same academic program affiliation as the current subject given as param
def get_same_program_subjs(curr_subj, curr_subj_program, map_prog_to_subj):
    return [subj for program, subjs in map_prog_to_subj.items() if program == curr_subj_program for subj in subjs if subj != curr_subj]

In [35]:
# Extract list of subjects with a different academic program affiliation as the current subject given as param
def get_diff_program_subjs(curr_program, map_prog_to_subj):
    return [subj for program, subjs in map_prog_to_subj.items() if program != curr_program for subj in subjs]

In [ ]:
# Calculate mean voxel activity of a subgroup of subjects given as param, based on a given time point and category_prefix
def calc_mean_subjs_data(time_point, category_prefix, relevant_subjs):
    if category_prefix == '':
        time_series = conf.FMRI_TIMESERIES_ALL_CATEGORIES
    else:
        time_series = conf.FMRI_TIMESERIES_ONE_CATEGORY

    mean_subjs_data = np.zeros((time_series, conf.N_VOXELS))

    for subj in relevant_subjs:
        
        cifti_path = f'{conf.ROOT_PATH}/{time_point}/{subj}/MNINonLinear/Results/movies_concat_data/{category_prefix}movies_concat_data_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii'
        load_cifti = nib.load(cifti_path)
        cifti_data = load_cifti.get_fdata()
        mean_subjs_data += cifti_data
        
    mean_subjs_data = mean_subjs_data / len(relevant_subjs)
    
    return mean_subjs_data

In [ ]:
# Extract atlas cifti file and number of regions inside it
ATLAS_CIFTI  = nib.load(conf.ATLAS_CIFTI_PATH).get_fdata()
N_REGIONS = int(np.max(ATLAS_CIFTI))

# Load behavioral data of fMRI participants
behav_data_df = pd.read_excel('/Volumes/homes/Maya/Guests/Amit/behav_data.xlsx')
behav_data_df = behav_data_df[~behav_data_df['subj'].astype(str).isin(conf.EXCLUDED)]
behav_data_df['subj_formatted'] = behav_data_df['subj'].astype(str).str.zfill(3)

# Create a dict for mapping program with it's corresponded affiliated subjects
map_prog_to_subj = behav_data_df.groupby('program')['subj_formatted'].apply(list).to_dict()

for time_point in conf.TIME_POINTS:

    for category_prefix in conf.SCANS_CATEGORIES_PREFIX:
        
        same_prog_corr_list = []
        diff_prog_corr_list = []
        
        map_prog_to_mean_diff = {}
        for prog in map_prog_to_subj.keys():
            diff_prog_subjs = get_diff_program_subjs(prog, map_prog_to_subj)
            print(f"compute mean {prog}")
            map_prog_to_mean_diff[prog] = calc_mean_subjs_data(time_point, category_prefix, diff_prog_subjs)
            
        for idx, row in behav_data_df.iterrows():
            subj = row['subj_formatted']
            
            print(f"Analyzing {time_point} {category_prefix} scan of subject {subj}")
            
            same_prog_subj_row = {'subject' : subj}
            diff_prog_subj_row = {'subject' : subj}

            cifti_path = f'{conf.ROOT_PATH}/{time_point}/{subj}/MNINonLinear/Results/movies_concat_data/{category_prefix}movies_concat_data_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii'
            load_cifti = nib.load(cifti_path)
            subj_i_data = load_cifti.get_fdata()

            # Mean of subjects' academic program
            same_prog_subjs = get_same_program_subjs(subj, row['program'], map_prog_to_subj)
            same_program_mean = calc_mean_subjs_data(time_point, category_prefix, same_prog_subjs)
            not_subject_program_mean = map_prog_to_mean_diff[row['program']]

            for region in range(1, N_REGIONS+1):
                timeseries_subj_i = get_region_mean(region, subj_i_data, ATLAS_CIFTI)
                timeseries_same_program = get_region_mean(region, same_program_mean, ATLAS_CIFTI)
                timeseries_diff_program = get_region_mean(region, not_subject_program_mean, ATLAS_CIFTI)

                # correlation of subject's neural activity in brain region with same program's mean activity
                region_same_program_corr, _ = sc.pearsonr(timeseries_subj_i, timeseries_same_program)
                same_prog_subj_row[str(region)] = np.arctanh(region_same_program_corr) #Fisher's z transformation

                # correlation of subject's neural activity in brain region with different program's mean activity
                region_diff_program_corr, _ = sc.pearsonr(timeseries_subj_i, timeseries_diff_program)
                diff_prog_subj_row[str(region)] = np.arctanh(region_diff_program_corr) #Fisher's z transformation
            
            same_prog_corr_list.append(same_prog_subj_row)
            diff_prog_corr_list.append(diff_prog_subj_row)

        pd.DataFrame(same_prog_corr_list).to_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'{time_point}_{category_prefix}same_prog_corr.csv'))
        pd.DataFrame(diff_prog_corr_list).to_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'{time_point}_{category_prefix}diff_prog_corr.csv'))
                

compute mean Engineering_Biology
019


In [ ]:
!pip install xlrd


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
